### Creating Modeled BQ Dataset

In [1]:
covid = "CovidGlobal_modeled"
!bq --location=US mk --dataset {covid}

Dataset 'fluid-etching-266722:CovidGlobal_modeled' successfully created.


### Creating a modeled table ConfirmedGlobal

In [7]:
%%bigquery
create table CovidGlobal_modeled.ConfirmedGlobal as
select Province_State, Country_Region, Lat, Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_staging.time_series_covid19_confirmed_global


""


ConfirmedGlobal does not have primary key, so we would need to create one as well as casting Lat into float

### Creating a modeled table ConfirmedUSA

In [3]:
%%bigquery
create table CovidGlobal_modeled.ConfirmedUSA as
select distinct UID, iso3, Admin2, Province_State, Country_Region, Lat, Long_, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_staging.time_series_covid19_confirmed_US

""


In [4]:
%%bigquery
select count(*) from CovidGlobal_modeled.ConfirmedUSA

,f0_
0,3261


In [5]:
%%bigquery
select count(distinct UID) from CovidGlobal_modeled.ConfirmedUSA

,f0_
0,3261


CovidGlobal_modeled.ConfirmedUSA has primary key

### Creating a modeled table DeathGlobal

%%bigquery
create table CovidGlobal_modeled.DeathGlobal as
select Province_State, Country_Region, Lat, Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_staging.time_series_covid19_deaths_global


DeathGlobal does not have primary key, so we would need to create one

### Creating a modeled table DeathUSA

In [9]:
%%bigquery
create table CovidGlobal_modeled.DeathUSA as
select distinct UID, iso3, Admin2, Province_State, Country_Region, Lat, Long_, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_staging.time_series_covid19_deaths_US

""


%%bigquery
select count(*) from CovidGlobal_modeled.DeathUSA

In [11]:
%%bigquery
select count(distinct UID) from CovidGlobal_modeled.DeathUSA

,f0_
0,3261


DeathUSA has primary key

### Creating a modeled table RecoveredGlobal

In [12]:
%%bigquery
create table CovidGlobal_modeled.RecoveredGlobal as
select Province_State, Country_Region, Lat, Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_staging.time_series_covid19_recovered_global


""


RecoveredGlobal does not have primary key, so we would need to create one.

### Checking for Foregin Keys

ConfirmedUSA should be one of the Parent Table

In [13]:
%%bigquery
select count(*)
from CovidGlobal_modeled.DeathUSA D left join CovidGlobal_modeled.ConfirmedUSA C
on C.UID = D.UID
where C.UID is null

,f0_
0,0


so DeathUSA has foregin key

### Performing Transforms

we will change the name of attributes and cast one of attributes to float from int to transform ConfirmedUSA

In [2]:
%%bigquery
create or replace table CovidGlobal_modeled.ConfirmedUSA_SQL_Final as
select "f6f733ed-acb8-4dc5-bd62-3052adbcfa9b" as country_id, UID, iso3 as terrority, Admin2 as location, Province_State as state, Country_Region as country, safe_cast(Lat as FLOAT64) as Lat, Long_ as Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_modeled.ConfirmedUSA

""


In [4]:
%%bigquery
select count(*) from CovidGlobal_modeled.ConfirmedUSA_SQL_Final

,f0_
0,3261


In [5]:
%%bigquery
select count(distinct uid) from CovidGlobal_modeled.ConfirmedUSA_SQL_Final

,f0_
0,3261


ConfirmedUSA_SQL_Final has primary key

In [9]:
%%bigquery
select count(*)
from CovidGlobal_modeled.ConfirmedUSA_SQL_Final U left join CovidGlobal_modeled.ConfirmedGlobal_SQL_Final G
on U.country_id = G.UID
where G.UID is null

,f0_
0,0


ConfirmedUSA_SQL_Final has foregin key

We will simply change the names of attributes of DeathUSA to make it easier to read

In [3]:
%%bigquery
create or replace table CovidGlobal_modeled.DeathUSA_SQL_Final as
select "f6f733ed-acb8-4dc5-bd62-3052adbcfa9b" as country_id, UID, iso3 as terrority, Admin2 as location, Province_State as state, Country_Region as country, Lat, Long_ as Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_modeled.DeathUSA

""


In [6]:
%%bigquery
select count(*) from CovidGlobal_modeled.DeathUSA_SQL_Final

,f0_
0,3261


In [7]:
%%bigquery
select count(distinct uid) from CovidGlobal_modeled.DeathUSA_SQL_Final

,f0_
0,3261


DeathUSA_SQL_Final has primary key

In [10]:
%%bigquery
select count(*)
from CovidGlobal_modeled.DeathUSA_SQL_Final D left join CovidGlobal_modeled.ConfirmedUSA_SQL_Final C
on D.country_id = C.country_id
where C.country_id is null

,f0_
0,0


DeathUSA_SQL_Final has foregin key

We will make a primary key for ConfirmedGlobal and cast one of the field into apportiate type.

In [11]:
%%bigquery
create or replace table CovidGlobal_modeled.ConfirmedGlobal_SQL_Final as
select generate_uuid() as uid, Province_State as state, Country_Region as country, safe_cast(Lat as FLOAT64) as Lat,Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_modeled.ConfirmedGlobal

""


In [39]:
%%bigquery
select count(*) from CovidGlobal_modeled.ConfirmedGlobal_SQL_Final

,f0_
0,266


In [40]:
%%bigquery
select count(distinct uid) from CovidGlobal_modeled.ConfirmedGlobal_SQL_Final

,f0_
0,266


ConfirmedGlobal has primary key

In [41]:
%%bigquery
create or replace table CovidGlobal_modeled.DeathGlobal_SQL_2 as
select Province_State as state, Country_Region as country, safe_cast(Lat as FLOAT64) as Lat,Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_modeled.DeathGlobal 

""


In [42]:
%%bigquery
create or replace table CovidGlobal_modeled.DeathGlobal_SQL_Final as
select C.uid, D.state, D.country, D.Lat, D.Long, D._3_24_20, D._3_25_20, D._3_26_20, D._3_27_20, D._3_28_20
from CovidGlobal_modeled.DeathGlobal_SQL_2 D join CovidGlobal_modeled.ConfirmedGlobal_SQL_Final C on D.country = C.country 
where (D.state = C.state) or (D.state is null and C.state is null)

""


In [43]:
%%bigquery
select count(*) from CovidGlobal_modeled.DeathGlobal_SQL_Final

,f0_
0,266


In [44]:
%%bigquery
select count(distinct uid) from CovidGlobal_modeled.DeathGlobal_SQL_Final

,f0_
0,266


DeathGlobal_SQL_Final has primary key

In [45]:
%%bigquery
select count(*)
from CovidGlobal_modeled.DeathGlobal_SQL_Final D left join CovidGlobal_modeled.ConfirmedGlobal_SQL_Final C
on C.UID = D.UID
where C.UID is null

,f0_
0,0


In [46]:
%%bigquery
select count(*)
from CovidGlobal_modeled.DeathGlobal_SQL_Final D left join CovidGlobal_modeled.ConfirmedGlobal_SQL_Final C
on C.UID = D.UID
where C.UID is null

,f0_
0,0



DeathGlobal_SQL_Final has multiple foregin key

In [29]:
%%bigquery
create or replace table CovidGlobal_modeled.RecoveredGlobal_SQL_2 as
select Province_State as state, Country_Region as country, safe_cast(Lat as FLOAT64) as Lat,Long, _3_24_20, _3_25_20, _3_26_20, _3_27_20, _3_28_20
from CovidGlobal_modeled.RecoveredGlobal 

""


In [38]:
%%bigquery
create or replace table CovidGlobal_modeled.RecoveredGlobal_SQL_Final as
select C.uid, R.state, R.country, R.Lat, R.Long, R._3_24_20, R._3_25_20, R._3_26_20, R._3_27_20, R._3_28_20
from CovidGlobal_modeled.RecoveredGlobal_SQL_2 R join CovidGlobal_modeled.ConfirmedGlobal_SQL_Final C on R.country = C.country 
where (R.state = C.state) or (R.state is null and C.state is null)

""


In [39]:
%%bigquery
select count(*) from CovidGlobal_modeled.RecoveredGlobal_SQL_Final

,f0_
0,251


In [40]:
%%bigquery
select count(distinct uid) from CovidGlobal_modeled.RecoveredGlobal_SQL_Final

,f0_
0,251


RecoveredGlobal has primary key

In [41]:
%%bigquery
select count(*)
from CovidGlobal_modeled.RecoveredGlobal_SQL_Final R left join CovidGlobal_modeled.ConfirmedGlobal_SQL_Final C
on C.UID = R.UID
where C.UID is null

,f0_
0,0


RecoveredGlobal has foregin key